In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.colab.data_table

In [ ]:
project_id = 'dataengineer-day1'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/Nested Queries**

In [ ]:
df = client.query('''
  SELECT
  action_hash,
  topic
FROM
  `bigquery-public-data.crypto_iotex.transaction_logs`
WHERE
  TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND action_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_iotex.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
LIMIT
  5''').to_dataframe()

df.head()

,action_hash,topic


**Joins**

In [52]:
df = client.query('''
  SELECT
  t.transaction_hash,
  t.sender,
  tb.block_hash
FROM (
  SELECT
    action_hash AS transaction_hash,
    sender
  FROM
    `bigquery-public-data.crypto_iotex.transaction_logs`) AS t
INNER JOIN (
  SELECT
    `hash` AS block_hash,
    producer
  FROM
    `bigquery-public-data.crypto_iotex.blocks`) AS tb
ON
  t.transaction_hash = tb.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,sender,block_hash


**Unions**

In [ ]:
df = client.query('''
  (SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_acs.cbsa_2011_5yr`
LIMIT
  10)
UNION DISTINCT
(SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_acs.cbsa_2012_3yr`
LIMIT
  5)''').to_dataframe()

df.head()

,geo_id
0,45220
1,10900
2,25420
3,34980
4,17460


**Aggregate Functions**

In [ ]:
df = client.query('''
  SELECT
  bl.`hash`,
  COUNT(bl.`hash`) AS total_transaction
FROM
  `bigquery-public-data.crypto_iotex.blocks` bl
WHERE
  TIMESTAMP_TRUNC(bl.timestamp, DAY) = TIMESTAMP("2019-10-07")
GROUP BY
  bl.`hash`''').to_dataframe()

df.head()

,hash,total_transaction
0,d0d699249e0c4c0a2384324ec9e2735f36cf4c6a22f2ce...,1
1,578bf40035396d71764bce5efef2842b2a4b527a5f1e40...,1
2,8d4754f57ff929be309d68b2adacbd3477134906f8bc2f...,1
3,03923db1fffe5bd938f5185c5450f6bc94a44ebb460111...,1
4,bdfef63e5219957b95df093dd13c268c92c4971860139c...,1


**Window Functions**

In [46]:
df = client.query('''
SELECT
  tl.action_hash,
  COUNT(tl.action_hash) AS total_transaction,
  ROW_NUMBER() OVER (ORDER BY COUNT(tl.action_hash) DESC) AS rank
FROM
  `bigquery-public-data.crypto_iotex.transaction_logs` tl
WHERE
  TIMESTAMP_TRUNC(tl.timestamp, DAY) = TIMESTAMP("2019-10-07")
GROUP BY
  tl.action_hash''').to_dataframe()

df.head()

,action_hash,total_transaction,rank
0,454dcff220901f538f0a675cb058703cf00d2cc5204e10...,104,1
1,31de9eff09adc862f918ec12a12c8c16e49ded0fda5082...,104,2
2,d2c20eedd5565b92664192617fc03dcf7a5256ad4466e5...,104,3
3,1bef6e57c8bd3b4e7f8aecace141ed3f1db49215d05e7c...,104,4
4,6bfa03244ee2e8b114af6e0fa206feded00cc87c9cfdcc...,104,5


**Common Table Expressions (CTEs)**

In [ ]:
df = client.query('''
WITH
  transaction_block AS (
  SELECT
    action_hash AS transaction_hash,
    sender
  FROM
    `bigquery-public-data.crypto_iotex.transaction_logs`
  WHERE
    action_hash = "6daee4f54630151308cdaa875e39c9b33a34c9d87afc462fe0a35d8f63df96a9")
SELECT
  transaction_block.transaction_hash,
  transaction_block.sender
FROM
  transaction_block
ORDER BY
  transaction_block.transaction_hash
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,sender
0,6daee4f54630151308cdaa875e39c9b33a34c9d87afc46...,io1g5ggy4k72a7m9svnpypsvylygjwm9jclupvwps
1,6daee4f54630151308cdaa875e39c9b33a34c9d87afc46...,io1g5ggy4k72a7m9svnpypsvylygjwm9jclupvwps


**Pivot**

In [ ]:
%%bigquery --project dataengineer-day1
WITH Country AS (
  SELECT 'Indonesia' as country, 207 as total_citizen, 'Q1' as quarter, 2020 as year UNION ALL
  SELECT 'Indonesia', 214, 'Q2', 2020 UNION ALL
  SELECT 'Indonesia', 310, 'Q3', 2020 UNION ALL
  SELECT 'Indonesia', 333, 'Q4', 2020 UNION ALL
  SELECT 'Indonesia', 354, 'Q1', 2021 UNION ALL
  SELECT 'Indonesia', 376, 'Q2', 2021 UNION ALL
  SELECT 'Malaysia', 123, 'Q1', 2020 UNION ALL
  SELECT 'Malaysia', 234, 'Q2', 2020 UNION ALL
  SELECT 'Malaysia', 345, 'Q1', 2021)
SELECT * FROM Country

Query is running:   0%|          |

Downloading:   0%|          |

,country,total_citizen,quarter,year
0,Indonesia,207,Q1,2020
1,Indonesia,214,Q2,2020
2,Indonesia,310,Q3,2020
3,Indonesia,333,Q4,2020
4,Indonesia,354,Q1,2021
5,Indonesia,376,Q2,2021
6,Malaysia,123,Q1,2020
7,Malaysia,234,Q2,2020
8,Malaysia,345,Q1,2021


In [ ]:
%%bigquery --project dataengineer-day1
WITH Country AS (
  SELECT 'Indonesia' as country, 207 as total_citizen, 'Q1' as quarter, 2020 as year UNION ALL
  SELECT 'Indonesia', 214, 'Q2', 2020 UNION ALL
  SELECT 'Indonesia', 310, 'Q3', 2020 UNION ALL
  SELECT 'Indonesia', 333, 'Q4', 2020 UNION ALL
  SELECT 'Indonesia', 354, 'Q1', 2021 UNION ALL
  SELECT 'Indonesia', 376, 'Q2', 2021 UNION ALL
  SELECT 'Malaysia', 123, 'Q1', 2020 UNION ALL
  SELECT 'Malaysia', 234, 'Q2', 2020 UNION ALL
  SELECT 'Malaysia', 345, 'Q1', 2021)
SELECT * FROM Country
PIVOT(SUM(total_citizen) FOR quarter IN ('Q1', 'Q2', 'Q3', 'Q4'))

Query is running:   0%|          |

Downloading:   0%|          |

,country,year,Q1,Q2,Q3,Q4
0,Indonesia,2020,207,214,310,333
1,Indonesia,2021,354,376,<NA>,<NA>
2,Malaysia,2020,123,234,<NA>,<NA>
3,Malaysia,2021,345,<NA>,<NA>,<NA>


**String Manipulation**

Concat

In [ ]:
%%bigquery --project dataengineer-day1
SELECT
  DISTINCT CONCAT(sender, recipient) AS address
FROM
  `bigquery-public-data.crypto_iotex.transaction_logs`
LIMIT
  2

Query is running:   0%|          |

Downloading:   0%|          |

,address
0,io1lvemm43lz6np0hzcqlpk0kpxxww623z5hs4mwuio1rq...
1,io1lvemm43lz6np0hzcqlpk0kpxxww623z5hs4mwuio15e...


**Date and Time**

In [ ]:
%%bigquery --project dataengineer-day1
SELECT
  TIMESTAMP_TRUNC(timestamp, DAY) AS tx_date
FROM
  bigquery-public-data.crypto_iotex.transaction_logs
WHERE
  timestamp <= CAST(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AS TIMESTAMP)
LIMIT
  3

Query is running:   0%|          |

Downloading:   0%|          |

,tx_date
0,2020-01-12 00:00:00+00:00
1,2020-01-12 00:00:00+00:00
2,2020-01-12 00:00:00+00:00


**Case Statement**

In [ ]:
%%bigquery --project dataengineer-day1
SELECT
  action_hash AS transaction_hash,
  height,
  CASE
    WHEN height >= 100000000 THEN 'High'
    ELSE 'Low'
END
  AS height_category
FROM
  bigquery-public-data.crypto_iotex.transaction_logs
LIMIT
  2

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_hash,height,height_category
0,4e1a1e415cc2f642e02cb19b11ac7ae902498d42c0ac78...,1328833,Low
1,5f55b58f89d250a0ba60bb973eed999d868a72a8abd8de...,1328833,Low


In [ ]:
%%bigquery --project dataengineer-day1
CREATE TEMP FUNCTION
  getCategory(arr ANY TYPE) AS (
    CASE
      WHEN arr >= 100000000000 THEN 'High'
      ELSE 'Low'
  END
    );
SELECT
  height,
  getCategory(height) AS category
FROM (
  SELECT
    height
  FROM
    bigquery-public-data.crypto_iotex.transaction_logs
  LIMIT
    3)

Query is running:   0%|          |

Downloading:   0%|          |

,height,category
0,5234505,Low
1,5234505,Low
2,5234505,Low


**Recursive Queries**

In [ ]:
%%bigquery --project dataengineer-day1
WITH
  RECURSIVE CTE_1 AS ( (
    SELECT
      1 AS iteration
    UNION ALL
    SELECT
      1 AS iteration)
  UNION ALL
  SELECT
    iteration + 1 AS iteration
  FROM
    CTE_1
  WHERE
    iteration < 3 )
SELECT
  iteration
FROM
  CTE_1
ORDER BY
  1 ASC

Query is running:   0%|          |

Downloading:   0%|          |

,iteration
0,1
1,1
2,2
3,2
4,3
5,3
